![Parked car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries, insurance is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance has requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to use simple Logistic Regression, identifying the single feature that results in the best-performing model, as measured by accuracy.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table (below) detailing the column names and descriptions below.

## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [23]:
# Import required libraries
library(readr)
library(dplyr)
library(glue)
library(yardstick)

# Load the data 
cars <- read_csv("car_insurance.csv")
head(cars)

Rows: 10000 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (19): id, age, gender, race, driving_experience, education, income, cred...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
569520,3,0,1,0,2,3,0.6290273,1,1,0,1,10238,12000,0,0,0,0,0
750365,0,1,1,0,0,0,0.3577571,0,0,0,0,10238,16000,0,0,0,0,1
199901,0,0,1,0,2,1,0.4931458,1,0,0,0,10238,11000,0,0,0,0,0
478866,0,1,1,0,3,1,0.2060129,1,0,0,1,32765,11000,0,0,0,0,0
731664,1,1,1,1,0,1,0.3883659,1,0,0,0,32765,12000,0,2,0,1,1
877557,2,0,1,2,2,3,0.6191274,1,1,0,1,10238,13000,0,3,0,3,0


In [24]:
# Check missing values
lapply(cars, function(x) sum(is.na(x)))


$id
[1] 0

$age
[1] 0

$gender
[1] 0

$race
[1] 0

$driving_experience
[1] 0

$education
[1] 0

$income
[1] 0

$credit_score
[1] 982

$vehicle_ownership
[1] 0

$vehicle_year
[1] 0

$married
[1] 0

$children
[1] 0

$postal_code
[1] 0

$annual_mileage
[1] 957

$vehicle_type
[1] 0

$speeding_violations
[1] 0

$duis
[1] 0

$past_accidents
[1] 0

$outcome
[1] 0

In [25]:
# Check the distribution of numerical variables
cars %>% 
select(credit_score, annual_mileage) %>%
summary()

  credit_score    annual_mileage 
 Min.   :0.0534   Min.   : 2000  
 1st Qu.:0.4172   1st Qu.:10000  
 Median :0.5250   Median :12000  
 Mean   :0.5158   Mean   :11697  
 3rd Qu.:0.6183   3rd Qu.:14000  
 Max.   :0.9608   Max.   :22000  
 NA's   :982      NA's   :957    

In [26]:
# Fill missing values with mean
cars$credit_score[is.na(cars$credit_score)] <- mean(cars$credit_score, na.rm = TRUE)
cars$annual_mileage[is.na(cars$annual_mileage)] <- mean(cars$annual_mileage, na.rm = TRUE)
# Verify if there is no null values
anyNA(cars)

[1] FALSE

In [27]:
# Storing features used for modeling and drop unused ones
features_df <- data.frame(features = names(subset(cars, select = -c(id, outcome))))

In [28]:
# Creating logistic regression model for each feature using for loop
for (col in features_df$features) {
	# Create a model
	model <- glm(glue('outcome ~ {col}'), data = cars, family = 'binomial')
	# Get prediction values of the model
	predictions <- round(fitted(model))
	# Measuring performance of each model using accuracy
    accuracy <- length(which(predictions == cars$outcome))/length(cars$outcome)
	      # Add accuracy to features_df
	      features_df[which(features_df$features == col), "accuracy"] <- accuracy
}

In [29]:
# Find the feature with largest accuracy
best_feature <- features_df$features[which.max(features_df$accuracy)]
best_accuracy <- max(features_df$accuracy)

# Create best_feature_df
best_feature_df <- data.frame(best_feature, best_accuracy)

# Check to see the results
best_feature_df

best_feature,best_accuracy
<chr>,<dbl>
driving_experience,0.7771
